In [4]:
import os, sys, gensim
from gensim.parsing.preprocessing import STOPWORDS

def lsisimilarity():
    def iter_documents(top_directory):
        """Iterate over all documents, yielding a document (=list of utf8 tokens) at a time."""
        for root, dirs, files in os.walk(top_directory):
            for file in filter(lambda file: file.endswith('.txt'), files):
                document = open(os.path.join(root, file)).read() # read the entire document, as one big string
                yield gensim.utils.tokenize(document, lower=True) # or whatever tokenization suits you

    class MyCorpus(object):
        def __init__(self, top_dir):
            self.top_dir = top_dir
            self.dictionary = gensim.corpora.Dictionary(iter_documents(top_dir))
            self.dictionary.filter_extremes(no_below=1, keep_n=35000) # check API docs for pruning params

        def __iter__(self):
            for tokens in iter_documents(self.top_dir):
                yield self.dictionary.doc2bow(tokens)

    corpus = MyCorpus('C:/Users/rohit/Documents/IPO Project/ipo1') # create a dictionary
    print corpus 
    a=corpus.dictionary
    gensim.corpora.MmCorpus.serialize('corpus_bow.mm', corpus)
    mm_corpus = gensim.corpora.MmCorpus('corpus_bow.mm')
    ##print(mm_corpus)
    clipped_corpus = gensim.utils.ClippedCorpus(mm_corpus,2261)  # use fewer documents during training, LDA is slow
    # ClippedCorpus new in gensim 0.10.1
    # copy&paste it from https://github.com/piskvorky/gensim/blob/0.10.1/gensim/utils.py#L467 if necessary (or upgrade your gensim)
    lda_model = gensim.models.LdaModel(clipped_corpus, num_topics=10, id2word=a, passes=4)

    ##%time 
    tfidf_model = gensim.models.TfidfModel(mm_corpus, id2word=a)

    lsi_model = gensim.models.LsiModel(tfidf_model[mm_corpus], id2word=a, num_topics=500)

    # cache the transformed corpora to disk, for use in later notebooks
    gensim.corpora.MmCorpus.serialize('corpus_tfidf.mm', tfidf_model[mm_corpus])
    gensim.corpora.MmCorpus.serialize('corpus_lsa.mm', lsi_model[tfidf_model[mm_corpus]])
    # gensim.corpora.MmCorpus.serialize('./data/wiki_lda.mm', lda_model[mm_corpus])

    tfidf_corpus = gensim.corpora.MmCorpus('corpus_tfidf.mm')
    # `tfidf_corpus` is now exactly the same as `tfidf_model[wiki_corpus]`
    ##print(tfidf_corpus)

    lsi_corpus = gensim.corpora.MmCorpus('corpus_lsa.mm')
    # and `lsi_corpus` now equals `lsi_model[tfidf_model[wiki_corpus]]` = `lsi_model[tfidf_corpus]`
    ##print(lsi_corpus)

    def tokenize(text):
        return [token for token in gensim.utils.simple_preprocess(text) if token not in STOPWORDS]

    from gensim.similarities import MatrixSimilarity, SparseMatrixSimilarity, Similarity

    # initialize the index
    index_dense = MatrixSimilarity(lsi_corpus, num_features=lsi_corpus.num_terms)

    index_sparse = SparseMatrixSimilarity(tfidf_corpus, num_features=tfidf_corpus.num_terms)

    index = Similarity('corpus.mm.index', lsi_corpus, num_features=lsi_corpus.num_terms)

    
    path = "C:/Users/rohit/Documents/IPO Project/ipo1"
    os.chdir(path)
    dirs = os.listdir(path)
    for file in dirs:
        print file
        file1 = open('C:/Users/rohit/Documents/IPO Project/ipofull_order.txt','a')
        file1.write(file)
        with open (file, "r") as myfile:
            text1=myfile.read()
            query = text1

            # vectorize the text into bag-of-words and tfidf
            query_bow = a.doc2bow(tokenize(query))
            query_tfidf = tfidf_model[query_bow]
            query_lsi = lsi_model[query_tfidf]


            index_sparse.num_best = 5
            string_tfidf = index_sparse[query_tfidf]

            # query the LSI index
            index_dense.num_best = 5
            string_lsi= index_dense[query_lsi]
            final_str= string_tfidf+string_lsi
            final_str=str(final_str)
            final_string1=''
            final_string1=final_string1+final_str
            file1 = open('C:/Users/rohit/Documents/IPO Project/ipofull_lsi.txt','a')
            file1.write(final_string1)
            ##file1.write(b)
            file1.close()


lsisimilarity()



1000278.txt
1000495.txt
1000753.txt
1000897.txt
1000925.txt
1001136.txt
1001193.txt
1001233.txt
1001279.txt
1001426.txt
1001915.txt
1002119.txt
1002276.txt
1002628.txt
1002636.txt
1002658.txt
1003213.txt
1003214.txt
1003233.txt
1003265.txt
1003504.txt
1003607.txt
1003608.txt
1003989.txt
1004605.txt
1004874.txt
1004913.txt
1004967.txt
1004968.txt
1005139.txt
1005653.txt
1005938.txt
1006459.txt
1006591.txt
1006835.txt
1006837.txt
1007209.txt
1007304.txt
1008653.txt
1008848.txt
1008946.txt
1009386.txt
1009532.txt
1009575.txt
1009626.txt
1009657.txt
1009667.txt
1009759.txt
1009769.txt
1009773.txt
1009822.txt
1009833.txt
1009863.txt
1009900.txt
1009988.txt
1010026.txt
1010247.txt
1010286.txt
1010655.txt
1010677.txt
1010724.txt
1010919.txt
1011255.txt
1011344.txt
1011418.txt
1011570.txt
1011584.txt
1011630.txt
1011632.txt
1011657.txt
1011693.txt
1011888.txt
1012019.txt
1012123.txt
1012270.txt
1012367.txt
1012369.txt
1012393.txt
1012394.txt
1012441.txt
1012459.txt
1012466.txt
1012480.txt
1012